# Example State Machine Usage

Shows use of (amongst other things)

 - Actions
 - Extended State Variables
 - Guards Conditions.

An action does something with an event and stores it (in this example, transforms the string 'pizza' to 'cooked 'pizza'.

It then waits for a 'cooked order' event, with payload of 'cooked pizza', before giving it to the customer.

In [38]:
from state_machine import StateMachine, State, Model, Event, PseudoState

In [39]:
receptionist = StateMachine( 'Receptionist' )

wait_for_phone      = receptionist.create_state('Wait For Phone')
take_order          = receptionist.create_state('Take Order')
enter_order         = receptionist.create_state('Enter Order')
wait_for_order      = receptionist.create_state('Wait For Order')
give_order_to_cust  = receptionist.create_state('Give Order')

wait_for_phone.add_transition_to(take_order, 'phone rings')
take_order.add_transition_to(enter_order, 'customer orders')

INFO:default:Creating state of type <class 'state_machine.state.State'>
INFO:default:Creating state of type <class 'state_machine.state.State'>
INFO:default:Creating state of type <class 'state_machine.state.State'>
INFO:default:Creating state of type <class 'state_machine.state.State'>
INFO:default:Creating state of type <class 'state_machine.state.State'>


When the 'enter_order' state is entered, the enter_order_action is called and stores 'cooked pizza'.

In [40]:
def enter_order_action(event, state):
    state.machine.vars['order']='cooked %s'%event.payload
    state.logger.info( 'Ordered %s',state.machine.vars['order'] )

enter_order.add_transition_to(wait_for_order)
enter_order.on_start=enter_order_action    

The guard, wait_for_order_guard, only lets transition pass if the payload is the same as the order variable.

In [41]:
def wait_for_order_action(event, state):
    state.logger.info('Wait for order action %s',state.machine.vars)
    
def wait_for_order_guard(event, state):
    if event.payload==state.machine.vars['order']:
        state.logger.info( 'Guard received %s, which is what it wants', event.payload)
        return True
    else:
        state.logger.info( 'Guard received %s, which it does not want', event.payload)
        
wait_for_order.on_start = wait_for_order_action    
wait_for_order.add_transition_to( give_order_to_cust, 'cooked order', guard=wait_for_order_guard)

In [42]:
def give_order_action(event, state):
    state.logger.info('Giving %s to customer',event.payload)

give_order_to_cust.on_start = give_order_action
give_order_to_cust.add_transition_to(wait_for_order)

In [43]:
receptionist.set_state(wait_for_phone)

evt = Event('phone rings')
receptionist.notify(evt)

evt = Event('customer orders', 'pizza')
receptionist.notify(evt)

evt = Event('cooked order', 'cooked lasagne')
receptionist.notify(evt)


evt = Event('cooked order', 'cooked pizza')
receptionist.notify(evt)


INFO:default:Starting state Take Order
INFO:default:Starting state Enter Order
INFO:default:Ordered cooked pizza
INFO:default:Starting state Wait For Order
INFO:default:Wait for order action {'order': 'cooked pizza'}
INFO:default:Guard received cooked pizza, which is what it wants
INFO:default:Starting state Give Order
INFO:default:Giving cooked pizza to customer
